In [1]:
import pandas as pd
import numpy as np 
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
import math
import sklearn

#Read data from csv
df_orig_data = pd.read_csv("data.csv")
#Drop CAX_ID column
df_orig_data = df_orig_data.iloc[:,1:]


In [2]:
#Select numerical and non-numerical features

In [3]:
numerical_col_names = list(df_orig_data.select_dtypes(include=[np.number]).columns.values)
non_numerical_col_names = list(df_orig_data.select_dtypes(exclude=[np.number]).columns.values)

In [4]:
#One-hot encoding for categorical variables

In [5]:
cat_vars = non_numerical_col_names
data = df_orig_data
for i in range(len(non_numerical_col_names)):
    cat_list = pd.get_dummies(data.loc[:,non_numerical_col_names[i]], prefix = non_numerical_col_names[i])
    data1 = pd.concat([data, cat_list], axis=1)
    data = data1
    
cat_vars = non_numerical_col_names
data_vars = data.columns.values.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]
data_final = data[to_keep]
data_final_vars = data.columns.values.tolist()
y = ['Dependent']
X = [i for i in data_final if i not in y]
data_final = data[X]
y_final = data[y]

In [6]:
#Perform RFE and Cross-Validation

In [7]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

result = []
for j in range(79): #len(list(data_final.columns.values))
    
    logreg = LogisticRegression()
    num_features = j+1
    rfe = RFE(logreg, num_features)
    rfe = rfe.fit(data_final, y_final)

    #Create a list of column names for features selected by RFE
    data_final_col_names = data_final.columns.values
    rfe_list = list(rfe.support_)
    col_names_final = []
    for i in range(len(data_final_col_names)):

        if(rfe_list[i] == True):
            col_names_final.append(data_final_col_names[i])


    X = data_final.loc[:,col_names_final]
    Y = y_final

    
    logreg = LogisticRegression()
    from pandas import ExcelWriter

    k = 5
    a = len(X)/k
    test_start = -a
    train_len = len(X)
    total_score = []
    for i in range(k):

        test_start = test_start + a
        test_end = test_start + a
        if train_len - test_end < a :
            test_end = train_len
    #     print start, end
        train_01_start = 0
        train_02_start = test_end
        train_01_end = test_start
        train_02_end = train_len

        if test_end == train_len :
            train_02_end = test_start

        X_Train_01 = X.iloc[train_01_start:train_01_end,:]
        X_Train_02 = X.iloc[train_02_start:train_02_end,:]
        X_Train = X_Train_01.append(X_Train_02,ignore_index=True)
        Y_Train_01 = Y.iloc[train_01_start:train_01_end,:]
        Y_Train_02 = Y.iloc[train_02_start:train_02_end,:]
        Y_Train = Y_Train_01.append(Y_Train_02,ignore_index=True)
        X_Test = X.iloc[test_start:test_end,:]
        Y_Test = Y.iloc[test_start:test_end,:]

        logreg.fit(X_Train, Y_Train)
        score = logreg.score(X_Test, Y_Test)
        total_score.append(score)

    score_overall = sum(total_score)/len(total_score)
    result.append([num_features,score_overall])
    print('Accuracy of logistic regression classifier over all sets: {:.2f}', score_overall)

C:\Users\stoian\Anaconda2\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.63565217391304341)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.53130434782608693)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.64399999999999991)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.52347826086956517)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.52817391304347827)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.58469565217391306)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.52817391304347827)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.53217391304347827)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.56660869565217387)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.56660869565217387)
('Accuracy of logistic regression classifier over all sets: {:.2f}', 0.59408695652173904)
('Accuracy

In [8]:
from sklearn import svm
clf = svm.SVC()

k = 5
a = len(X)/k
test_start = -a
train_len = len(X)
total_score = []
for i in range(k):
    
    test_start = test_start + a
    test_end = test_start + a
    if train_len - test_end < a :
        test_end = train_len

    train_01_start = 0
    train_02_start = test_end
    train_01_end = test_start
    train_02_end = train_len
    
    if test_end == train_len :
        train_02_end = test_start
    
    print (train_01_start, train_01_end)
    print (train_02_start, train_02_end)
    
    X_Train_01 = X.iloc[train_01_start:train_01_end,:]
    X_Train_02 = X.iloc[train_02_start:train_02_end,:]
    X_Train = X_Train_01.append(X_Train_02,ignore_index=True)
    Y_Train_01 = Y.iloc[train_01_start:train_01_end,:]
    Y_Train_02 = Y.iloc[train_02_start:train_02_end,:]
    Y_Train = Y_Train_01.append(Y_Train_02,ignore_index=True)
    X_Test = X.iloc[test_start:test_end,:]
    Y_Test = Y.iloc[test_start:test_end,:]
    
    clf.fit(X_Train, Y_Train) 
    score = clf.score(X_Test, Y_Test)
    total_score.append(score)
    print('Accuracy of SVM classifier on test set: {:.2f}',score)

score_overall = sum(total_score)/len(total_score)
print('Accuracy of SVM classifier over all sets: {:.2f}', score_overall)
    

(0, 0)
(46, 234)
('Accuracy of SVM classifier on test set: {:.2f}', 0.47826086956521741)
(0, 46)
(92, 234)
('Accuracy of SVM classifier on test set: {:.2f}', 0.54347826086956519)
(0, 92)
(138, 234)
('Accuracy of SVM classifier on test set: {:.2f}', 0.47826086956521741)
(0, 138)
(184, 234)
('Accuracy of SVM classifier on test set: {:.2f}', 0.43478260869565216)
(0, 184)
(234, 184)
('Accuracy of SVM classifier on test set: {:.2f}', 0.35999999999999999)
('Accuracy of SVM classifier over all sets: {:.2f}', 0.45895652173913043)
